In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import nn

In [3]:
import torch as th

In [4]:
x = th.rand((5, 3))
f = nn.HyperbolicLinearLinear(3, 2)

In [5]:
f(x)

M is  Parameter containing:
tensor([[0.7218, 0.1506],
        [0.5153, 0.0240],
        [0.7344, 0.7376]], requires_grad=True)
x is  tensor([[0.8792, 0.7054, 0.1159],
        [0.0201, 0.3425, 0.2145],
        [0.6103, 0.9871, 0.9232],
        [0.4629, 0.6136, 0.8344],
        [0.6640, 0.8427, 0.6210]])
tensor([[1.0831, 0.2347],
        [0.3485, 0.1695],
        [1.6271, 0.7965],
        [1.2631, 0.6999],
        [1.3695, 0.5782]], grad_fn=<MmBackward>)
tensor([[2.7227, 1.2420]], grad_fn=<NormBackward1>)
tensor([[1.3420, 1.6358, 1.4120]])


RuntimeError: The size of tensor a (2) must match the size of tensor b (3) at non-singleton dimension 1

In [6]:
from non_tensor_ops import *

In [48]:
emb_dim = 20
v1 = np_random.uniform(-0.5, 0.5, emb_dim).astype(np.float32)
v2 = np_random.uniform(-0.5, 0.5, emb_dim).astype(np.float32)
v3 = np_random.uniform(-0.5, 0.5, (10, emb_dim)).astype(np.float32)
v1 = v1 * 0.59999 / LA.norm(v1)
v2 = v2 * 0.99 / LA.norm(v2)
v3 = np.stack((v1, v2))

In [49]:
M = np.random.rand(emb_dim, 5).astype(np.float32)

In [50]:
mob_mat_mul(M, v3, 1)

Mx norm1:  1.4754727
x norm  [0.59999 0.99   ]
Norm factor:  [2.459162  1.4903765]
inner2:  [0.63435405 0.67724097]
Mx:  [[-0.00241915  0.3175991   0.22756176  0.28619313  0.29497212]
 [-0.8694726  -0.32169852 -0.83165216 -0.53890795 -0.1179212 ]]


ValueError: operands could not be broadcast together with shapes (2,) (2,5) 

In [59]:
def mob_mat_mul(M, x, c):
    Mx = x @ M
    MX_norm = LA.norm(Mx.T, axis=0)
    x_norm = LA.norm(x.T, axis=0)
    print('Mx: ', Mx)
    print('Mx norm1: ',MX_norm)
    print('x norm ', x_norm)
    sqrt_recip = 1. / np.sqrt(c)
    norm_factor = MX_norm / x_norm
    print('Norm factor: ', norm_factor)
    inner1 = np.tanh(norm_factor * np.arctanh(np.sqrt(c) * x_norm)) / MX_norm
    inner2 = sqrt_recip * inner1
    inner2 = inner2.reshape(-1, 1)
    print('inner2: ', inner2)
    print('Mx: ', Mx)
    return  inner2 * Mx

In [60]:
print('Ans1: ',mob_mat_mul(M, v1, 1))
print()
print('Ans2: ',mob_mat_mul(M, v2, 1))
print()
print('Ans3: ',mob_mat_mul(M, v3, 1))

Mx:  [-0.00241917  0.3175991   0.22756174  0.28619313  0.29497212]
Mx norm1:  0.5670754
x norm  0.59999
Norm factor:  0.9451414
inner2:  [[1.01414763]]
Mx:  [-0.00241917  0.3175991   0.22756174  0.28619313  0.29497212]
Ans1:  [-0.00245339  0.32209235  0.2307812   0.29024208  0.29914528]

Mx:  [-0.8694726  -0.32169852 -0.83165216 -0.538908   -0.11792122]
Mx norm1:  1.3621472
x norm  0.99
Norm factor:  1.3759062
inner2:  [[0.73312695]]
Mx:  [-0.8694726  -0.32169852 -0.83165216 -0.538908   -0.11792122]
Ans2:  [-0.6374338  -0.23584585 -0.6097066  -0.395088   -0.08645122]

Mx:  [[-0.00241915  0.3175991   0.22756176  0.28619313  0.29497212]
 [-0.8694726  -0.32169852 -0.83165216 -0.53890795 -0.1179212 ]]
Mx norm1:  [0.5670754 1.3621472]
x norm  [0.59999 0.99   ]
Norm factor:  [0.9451414 1.3759062]
inner2:  [[1.0141476 ]
 [0.73312694]]
Mx:  [[-0.00241915  0.3175991   0.22756176  0.28619313  0.29497212]
 [-0.8694726  -0.32169852 -0.83165216 -0.53890795 -0.1179212 ]]


ValueError: operands could not be broadcast together with shapes (2,) (2,5) 

In [31]:
v3.T

array([[ 0.07412792, -0.25206652],
       [ 0.03832465, -0.04804948],
       [ 0.17992742, -0.01423273],
       [ 0.15613668,  0.19928516],
       [ 0.02445264, -0.18166915],
       [ 0.04639031,  0.35277745],
       [-0.13094404, -0.14644618],
       [-0.06186415, -0.04273972],
       [-0.1602464 , -0.03780794],
       [ 0.08079259, -0.01691131],
       [ 0.13536818,  0.33811027],
       [ 0.11236496,  0.24459991],
       [ 0.03893559, -0.14255436],
       [-0.18380708, -0.21330747],
       [-0.16104414,  0.12910773],
       [-0.05301902,  0.36234128],
       [-0.20518638,  0.21502838],
       [-0.2060926 , -0.07201783],
       [-0.18662345,  0.33805156],
       [ 0.1610599 ,  0.3718336 ]], dtype=float32)

In [27]:
LA.norm(v3)

1.1576217

In [28]:
LA.norm(v3, axis=0)

array([0.2627403 , 0.06146162, 0.18048947, 0.25316644, 0.18330741,
       0.35581455, 0.19645058, 0.07519213, 0.16464613, 0.08254354,
       0.36420202, 0.26917467, 0.14777593, 0.28157613, 0.20640741,
       0.3661997 , 0.29721817, 0.21831337, 0.38614395, 0.40521663],
      dtype=float32)

In [30]:
LA.norm(v3, axis=1)

array([0.59999   , 0.98999995], dtype=float32)